In [1]:
import pandas as pd

reviews = pd.read_json('data/yolosac-reviews.json')

reviews.head()

,text,rating,gmap_id
0,Helped me alot. If it was for her suggesting m...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed
1,Always leave feeling refreshed...polite and re...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed
2,Professional massage place. The ladies know w...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed
3,This place was pleasing to me. Very nice staff...,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed
4,Tough lady loves great pressure massage. Woohoo.,5.0,0x809ad993cd15cc69:0x38291ec9a11f59ed


In [46]:
# sample bigram classification from medium website
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.util import ngrams

# Function to extract bigrams from a text
def extract_bigrams(text):
    return list(ngrams(text.split(), 2))

gmap_id = "0x809ad68a4b531825:0x635fa3c8560daf3c"
some = reviews.loc[(reviews['gmap_id'] == gmap_id)]
rev = [(t, "positive" if r > 3 else "negative") for t, r in zip(some.text.tolist(), some.rating.tolist())]
print(rev)

# Combine the reviews into a single dataset

# Split the data into training and testing sets
training_data = rev[:int(0.8 * len(rev))]
testing_data = rev[int(0.8 * len(rev)):]

# Extract bigrams from the reviews and create a feature set
features = [(dict(extract_bigrams(review)), label) for (review, label) in training_data]

# Train a Naive Bayes classifier on the feature set
classifier = NaiveBayesClassifier.train(features)

# Evaluate the classifier on the testing data
accuracy = nltk.classify.accuracy(classifier, [(dict(extract_bigrams(review)), label) for (review, label) in testing_data])
print("Accuracy:", accuracy)

# Predict the sentiment of a new review
review = "good"
bigrams = extract_bigrams(review)
sentiment = classifier.classify(dict(bigrams))
print("Sentiment:", sentiment)

[('They have a well layed out menu board and the cashier gave some helpful suggestions. Ended up ordering the Supreme Cheese Steak and a side salad.\nTasty sandwich & basic ceaser prepared quickly.', 'positive'), ("Best massage I've had, which included great back walking and deep tissue. I've been here several times. Each time has been better than the last. I'm not sure why some of the other reviewers had a bad experience, but have to say I have not. I simply schedule the appointment and they've been ready for me every time.", 'positive'), ("I've been getting massages about once a week for the past 16 years.  My massage tonight was arguably the worst massage I've ever received.  I asked her (Maria) how long she had been doing massage and she told me 2 weeks.  Seems about right....ridiculous.  Then when I tipped her (which I REALLY didn't want to do in the first place) she had the nerve to ask me for a bigger tip.  Unless you want to waste your money on a worthless massage, I wouldn't r

In [4]:
review = "Great food and our waiter, she was a very friendly super awesome waiter."
from nltk.util import ngrams
import pandas as pd

def getNgrams(n):
    return pd.Series(ngrams(review.split(), n))

df = pd.DataFrame({
    "1-gram" : getNgrams(1),
    "2-gram" : getNgrams(2),
    "3-gram" : getNgrams(3),
    "4-gram" : getNgrams(4),
})
print(review)
df

Great food and our waiter, she was a very friendly super awesome waiter.


,1-gram,2-gram,3-gram,4-gram
0,"(Great,)","(Great, food)","(Great, food, and)","(Great, food, and, our)"
1,"(food,)","(food, and)","(food, and, our)","(food, and, our, waiter,)"
2,"(and,)","(and, our)","(and, our, waiter,)","(and, our, waiter,, she)"
3,"(our,)","(our, waiter,)","(our, waiter,, she)","(our, waiter,, she, was)"
4,"(waiter,,)","(waiter,, she)","(waiter,, she, was)","(waiter,, she, was, a)"
5,"(she,)","(she, was)","(she, was, a)","(she, was, a, very)"
6,"(was,)","(was, a)","(was, a, very)","(was, a, very, friendly)"
7,"(a,)","(a, very)","(a, very, friendly)","(a, very, friendly, super)"
8,"(very,)","(very, friendly)","(very, friendly, super)","(very, friendly, super, awesome)"
9,"(friendly,)","(friendly, super)","(friendly, super, awesome)","(friendly, super, awesome, waiter.)"
